In [1]:
# %pip install pypsa
# %pip install flask
# %pip install pandas
# %pip install numpy

In [2]:
%reset -f
import os
import pypsa
import warnings
import threading as th
import webbrowser as wb
import pandas as pd
import numpy as np
import pypsabackground as pyp
import net_writer as nw
from pathlib import Path

# Set pandas options
pd.set_option('display.max_columns', None)
pd.options.plotting.backend = 'plotly'

# Configure warnings
warnings.filterwarnings("ignore", category=UserWarning, module='openpyxl')

network = pypsa.Network()

In [3]:
'''________________________________
Set the Time Period for the Network
________________________________'''

timePeriod = {
    "startDate": "2025-01-01 00:00",
    "yearAmount": 1
}

In [ ]:
'''____________________________________________________________
Set the path for Raw Data in Excel files for Folder Management:
_______________________________________________________________
(Do not change folders unless necessary)
(Move Input Data to their respective Folders before Running)'''

rawFolder = Path("RawData")  # Raw Data is in this Folder
rawData = {
    "Plant": {
        "Folder": "PlantData",       # Keep Folders the same for Convenience
        "File": "AfricaModel_V4",                                            # Change File Names for new Data
        "Sheet": "HydroAfrica"
    },
    "Demand": {
        "Folder": "DemandData",
        "File": "AfricaDemandData_V1"
    },
    "River": {
        "Folder": "RiverFlow",
        "File": "AfricaRiverData_V1"
    }
}

In [ ]:
'''__________________________________________
Set the path for Data Exporting as CSV files:
__________________________________________'''

exportFolder = Path("ExportCSV")
exportData = {
    "Raw" : {
        "Folder" : "RawExport",
        "File" : "AfricaRawData"
    },
    "Result" : {
        "Folder" : "ResultExport",
        "File" : "AfricaResultData"
    }
}

In [ ]:
'''_______________________________________________________________
Networks for the Webpage to Read are Saved in this Folder:
__________________________________________________________________
(Do not change otherwise the Webpage will not see the Networks)'''

saveNetFolder = Path("SavedNetworks")

In [4]:
'''__________________________
Create Folders for Exporting:
__________________________'''

saveNetFolder.mkdir(parents=True, exist_ok=True)
exportFolder.mkdir(parents=True, exist_ok=True)

In [5]:
'''___________________________________________________
This Takes the Data Inputs Above and makes the Network
___________________________________________________'''

# Automatic Folder Pathing for Above Data Inputs
plantPath = pyp.get_path(rawData, rawFolder, "Plant", "File")
riverPath = pyp.get_path(rawData, rawFolder, "River", "File")
demandPath = pyp.get_path(rawData, rawFolder, "Demand", "File")
start_date = pd.Timestamp(timePeriod["startDate"])
end_date = start_date + pd.DateOffset(years=timePeriod["yearAmount"]) - pd.Timedelta(hours=1)
snapshots = pd.date_range(start_date,end_date , freq="h")

# Selects a Solver
select_solver = pyp.solver_selected() #'highs'

# Create network
network = pypsa.Network()
network.set_snapshots(snapshots)

# Import Static, Plant, River and then Demand Data
pyp.import_from_excel(network, f"{str(plantPath)}.xlsx")
with pd.ExcelFile(f"{str(plantPath)}.xlsx") as xls:
    plant_data_df = pd.read_excel(xls, sheet_name=rawData["Plant"]["Sheet"], comment="#")
update_components = plant_data_df.apply(lambda row: pyp.add_plant_data(network, row), axis=1)
pyp.import_river_inflows(network, f"{str(riverPath)}.csv")
pyp.import_demand(network, f"{str(demandPath)}.csv")

Solver highs succeeded!
Selected solver: highs
Error adding link for 0: cannot convert float NaN to integer
Error adding link for 6: cannot convert float NaN to integer
Error adding link for 8: cannot convert float NaN to integer
Error adding link for 9: cannot convert float NaN to integer
Error adding link for 10: cannot convert float NaN to integer
Error adding link for 11: cannot convert float NaN to integer
Error adding link for 12: cannot convert float NaN to integer
Error adding link for 18: cannot convert float NaN to integer
Error adding link for 22: cannot convert float NaN to integer
Error adding link for 26: cannot convert float NaN to integer
Error adding link for 32: cannot convert float NaN to integer
Error adding link for 33: cannot convert float NaN to integer


In [6]:
'''___________________________________
Export Raw Data as csv files if needed
(Uncomment [Ctrl+/] to Export)
___________________________________'''

# exportRawPath = pyp.get_path(exportData, exportFolder, "Raw", "File")
# newRaw = pyp.get_next_filename(exportFolder / exportData['Raw']['Folder'], exportData['Raw']['File'], 'csv')
# network.export_to_csv_folder(exportFolder / exportData['Raw']['Folder'] / newRaw)
# rawFullPath = exportFolder / newRaw
# print(f"Raw saved to: {rawFullPath}")

'\n______________________________________\nExport Raw Data as csv files if needed\n______________________________________\n'

In [7]:
select_solver = pyp.solver_selected()
network.optimize(solver_name=select_solver)

Solver highs succeeded!
Selected solver: highs


INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io:Writing objective.
Writing binary variables.: 100%|██████████| 3/3 [00:00<00:00,  3.75it/s]
INFO:linopy.io: Writing time: 18.72s
INFO:linopy.solvers:Log file at C:\Users\dazzc\AppData\Local\Temp\highs.log
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 1322775 primals, 2119907 duals
Objective: 3.35e+10
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-ext-p-lower, Generator-ext-p-upper, Generator-p_set, Link-com-p-lower, Link-com-p-upper, Link-com-transition-start-up, Link-com-transition-shut-down, Link-com-status-min_up_time_must_stay_up, Link-com-p0-ramp_limit_up, Link-com-p0-ramp_limit_down, Store-fix-e-lower, Store-fix-e-upper, Store-energy_balance were not assigned to the network.


('ok', 'optimal')

In [8]:
'''______________________________________
Export Result Data as csv files if needed
______________________________________'''

# exportResultPath = pyp.get_path(exportData, exportFolder, "Result", "File")
# newResult = pyp.get_next_filename(exportFolder / exportData['Result']['Folder'], exportData['Result']['File'], 'csv')
# network.export_to_csv_folder(exportFolder / exportData['Result']['Folder'] / newResult)
# resultFullPath = exportFolder / newResult
# print(f"Result saved to: {resultFullPath}")

'\n_________________________________________\nExport Result Data as csv files if needed\n_________________________________________\n'

In [9]:
'''______________________________________________________________
Network Exported as .h5 File that net_writer Can Read and Display
______________________________________________________________'''

newNet = pyp.get_next_filename(str(saveNetFolder), "network", 'h5')
netFullPath = saveNetFolder / newNet
network.export_to_hdf5(netFullPath)
print(f"Network saved to: {netFullPath}")

INFO:pypsa.io:Exported network 'network5.h5' contains: links, stores, buses, carriers, generators, loads


Network saved to: SavedNetworks\network5.h5


In [10]:
'''__________________________________________________________
Run net_writer.py to start the Localhost and Open the Webpage
__________________________________________________________'''

def run_flask():
    nw.app.run(port=5000)
flask_thread = th.Thread(target=run_flask)
flask_thread.start()
wb.open("http://127.0.0.1:5000/")

 * Serving Flask app 'net_writer'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


True

INFO:werkzeug:127.0.0.1 - - [23/Oct/2024 11:36:29] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Oct/2024 11:36:29] "GET /static/css/styles.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [23/Oct/2024 11:36:29] "GET /static/js/script.js HTTP/1.1" 304 -
INFO:pypsa.io:Imported network network1.h5 has buses, carriers, generators, links, loads, stores
INFO:werkzeug:127.0.0.1 - - [23/Oct/2024 11:36:37] "POST /load_network HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Oct/2024 11:36:38] "GET /get_components HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Oct/2024 11:38:49] "GET /get_data/Generator/all/static HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Oct/2024 11:38:49] "GET /get_data/Generator/all/static HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Oct/2024 11:38:54] "GET /get_data/Link/all/static HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Oct/2024 11:38:54] "GET /get_data/Link/all/static HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Oct/2024 11:38:55] "GET /get_att